In [ ]:
#  Stand Count Analysis

# Libraries

from datetime import datetime, timedelta
import os
import pandas as pd
import seaborn as sns
import sentinelhub
from sentinelhub import SHConfig
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.colors import PowerNorm
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import numpy as np
import geopandas as gpd
import sklearn
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import load_model
import json
import memory_profiler
import rasterio
from rasterio.mask import mask as rio_mask
from shapely.geometry import Point,Polygon
from osgeo import gdal
import shutil
from sentinelhub import (
        CRS,
        BBox,
        DataCollection,
        DownloadRequest,
        MimeType,
        MosaickingOrder,
        SentinelHubDownloadClient,
        SentinelHubRequest,
        bbox_to_dimensions,
)


def standcount(org_name,crop_name,sowing_date,service_name,analysis_date,polygon_c,input_spacing):

    with open('/content/drive/MyDrive/internship/B - with GDAL/python_services_config.json','r') as config_file:
        get_configuration = json.load(config_file)

    config_data_stand_count=get_configuration['potato_stand_count']
    config_data_main=get_configuration['main']

    config = SHConfig()

    config.sh_client_id = config_data_main["sh_client_id"]
    config.sh_client_secret = config_data_main["sh_client_secret"]
    config.save()

    today=datetime.now()
    save_date=today.strftime("%Y-%m-%d-%H-%m-%s")

    home_directory = os.path.expanduser("~")
    cc_output_directory = os.path.join(home_directory,config_data_stand_count['output_folder'])
    os.makedirs(cc_output_directory, exist_ok=True)

    # date
    def extract_start_date(analysis_date):
        e=datetime.strptime(analysis_date,'%Y-%m-%d')
        s=e-timedelta(7)
        start_date=s.strftime('%Y-%m-%d')
        return start_date

    start_date=extract_start_date(analysis_date)
    end_date=analysis_date

    # extract the spacing format 25*26m2
    number=input_spacing.split("*")
    num1=float(number[0])
    num2=float(number[1])
    spacing=num1+num2

    # boudning box
    min_x = min(p[0] for p in polygon_c)
    max_x = max(p[0] for p in polygon_c)
    min_y = min(p[1] for p in polygon_c)
    max_y = max(p[1] for p in polygon_c)


    bounding_box = (min_x, min_y, max_x, max_y)
    resolution = 10
    betsiboka_bbox = BBox(bbox=bounding_box, crs=CRS.WGS84)
    betsiboka_size = bbox_to_dimensions(betsiboka_bbox, resolution=resolution)

    # print(f"Image shape at {resolution} m resolution: {betsiboka_size} pixels")

    evalscript_all_bands = """
        //VERSION=3
        function setup() {
            return {
                input: [{
                    bands: ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"],
                    units: "DN"
                }],
                output: {
                    bands: 13,
                    sampleType: "INT16"
                }
            };
        }

        function evaluatePixel(sample) {
            return [sample.B01,
                    sample.B02,
                    sample.B03,
                    sample.B04,
                    sample.B05,
                    sample.B06,
                    sample.B07,
                    sample.B08,
                    sample.B8A,
                    sample.B09,
                    sample.B10,
                    sample.B11,
                    sample.B12];
        }
    """

    request_all_bands = SentinelHubRequest(
        evalscript=evalscript_all_bands,
        data_folder=config_data_main["data_output_folder"],
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L1C,
                time_interval=(str(start_date),str(end_date)),
                mosaicking_order=MosaickingOrder.LEAST_CC,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=betsiboka_bbox,
        size=betsiboka_size,
        config=config,
    )

    try:

        all_bands_response = request_all_bands.get_data()

        # stacked image
        # %%time
        all_bands_img = request_all_bands.get_data(save_data=True)

        for folder, _, filenames in os.walk(request_all_bands.data_folder):
            for filename in filenames:
                file_path = os.path.join(folder, filename)

                if filename.endswith(".tif") or filename.endswith(".tiff"):

                    try:
                      # clip the raster for getting the image with nan pixels
                      selected_bands=[2,3,4,8]
                      def clip_and_remove_zero_pixels(file_a,path_b,polygon_c):
                          with rasterio.open(file_a) as src:
                              polygon = Polygon(polygon_c)

                              out_image, out_transform = rio_mask(src, [polygon], crop=True)

                              out_meta = src.meta.copy()
                              out_meta.update({"driver": "GTiff",
                                              "height": out_image.shape[1],
                                              "width": out_image.shape[2],
                                              "transform": out_transform})

                          clipped_raster_path = config_data_main["data_output_folder"]+"file.tiff"
                          with rasterio.open(clipped_raster_path, "w", **out_meta) as dest:
                              dest.write(out_image)

                          src_ds = gdal.Open(clipped_raster_path)
                          if src_ds is None:
                              print("Failed to open the clipped raster file.")
                              return

                          cols = src_ds.RasterXSize
                          rows = src_ds.RasterYSize
                          bands = src_ds.RasterCount

                          driver = gdal.GetDriverByName("GTiff")
                          dst_ds = driver.Create(path_b, cols, rows, bands, gdal.GDT_Float32)

                          dst_ds.SetProjection(src_ds.GetProjection())
                          dst_ds.SetGeoTransform(src_ds.GetGeoTransform())

                          for band_idx in range(1, bands + 1):
                              band = src_ds.GetRasterBand(band_idx)
                              band_array = band.ReadAsArray()

                              mask_array = (band_array == -99999)

                              masked_band_array = np.where(mask_array, np.nan, band_array)

                              dst_band = dst_ds.GetRasterBand(band_idx)
                              dst_band.WriteArray(masked_band_array)
                          src_ds = None
                          dst_ds = None
                          os.remove(clipped_raster_path)

                      clip_and_remove_zero_pixels(file_path,file_path,polygon_c)

                      # main functionality on file path
                      selected_bands=[2,3,4,8]
                      with rasterio.open(file_path) as src:
                          bands_data = src.read(selected_bands)
                          width, height = src.width, src.height
                          count = len(selected_bands)
                          flat_data = bands_data.reshape((count, -1)).T
                          column_names = [f'Band_{band}' for band in selected_bands]
                          transform = src.transform
                          srs = src.crs
                          rows, cols = src.height, src.width
                          y_coords, x_coords = [y for y in range(rows) for _ in range(cols)], [x for _ in range(rows) for x in range(cols)]
                          pixel_coords = list(zip(y_coords, x_coords))
                          lonlat_coords = [src.transform * (x, y) for y, x in pixel_coords]
                          pixel_values_with_coords = [(lon, lat, *values) for (lon, lat), values in zip(lonlat_coords, flat_data)]
                          column_names = ['Longitude', 'Latitude'] + column_names
                          df = pd.DataFrame(pixel_values_with_coords, columns=column_names)

                          for column in df.columns:
                              df = df[df[column] != 0]

                          # print(df.shape)
                          df=df.reset_index()
                          df=df.drop(columns={"index"})

                          def ndvi(row):
                              return (row['Band_8']-row['Band_4'])/(row['Band_8']+row['Band_4'])

                          def gci(row):
                              return (row['Band_8']-row['Band_3'])/(row['Band_8']+row['Band_3'])

                          def lai(row):
                              return (row['Band_8']-row['Band_4'])/((row['Band_8']+row['Band_4'])*2.5)

                          def evi(row):
                              gain_factor = 2.5
                              offset = 1.0
                              denominator=row['Band_8'] + 6 * row['Band_4'] - 7.5 * row['Band_2'] + offset

                              if denominator==0:
                                  evi=0
                                  return evi
                              else:
                                  evi = gain_factor * (row['Band_8']-row['Band_4']) / (denominator)
                                  return evi


                          df['ndvi']=df.apply(ndvi,axis=1)
                          df['gci']=df.apply(gci,axis=1)
                          df['lai']=df.apply(lai,axis=1)
                          df['evi']=df.apply(evi,axis=1)

                          df.columns=["Longitude","Latitude","Band1","Band2","Band3","Band4","NDVI","GCI","LAI","EVI"]

                          X=df.iloc[:,2:]

                          X[np.isinf(X)] = np.finfo(np.float32).max
                          X[np.isnan(X)] = 0

                          scaler = StandardScaler()
                          X_test = scaler.fit_transform(X)

                          #load the model
                          # classes - 0 - canopy cover , 1 - no canopy

                          #predictions
                          ann_sc = tf.keras.models.load_model(config_data_stand_count["sc_ml_model"])

                          # predictions
                          predictions_ann=ann_sc.predict(X_test)
                          argument=np.argmax(predictions_ann,axis=1)
                          total_1=np.count_nonzero(argument==1)
                          total_0=np.count_nonzero(argument==0)


                          pixel_resolution=10*10
                          total_pixel_counts=total_0+total_1
                          total_area_m2=total_pixel_counts*pixel_resolution
                          total_area_acres=total_area_m2/4046.86
                          canopy_cover_area_m2=total_0*pixel_resolution
                          canopy_cover_area_acres=canopy_cover_area_m2/4046.86

                          #calculations

                          def area_converter(A):
                            b=str(A).split('.')
                            num1=b[0]+'.'+b[1][:3]
                            if float(b[1][1][:3]) < 5.0:
                              num1=b[0]+'.'+b[1][:3]
                              num1=float(num1)
                              num1=np.round(num1,2)
                              return num1
                            else:
                              num1=b[0]+'.'+b[1][:3]
                              num2=num1[:num1.find(num1.replace(num1[-1]," "))]
                              num2=num2.strip()
                              num2=float(num2)
                              return num2

                          # back up -- >>
                          # def restrict_round(c):
                          #   c=str(c)
                          #   b=c[c.find('.')-c.find('.'):c.find('.')+2]
                          #   b=float(b)
                          #   return b

                          canopy_cover_area_acres=area_converter(canopy_cover_area_acres)
                          stand_count=canopy_cover_area_m2/spacing
                          stand_count=int(np.round(stand_count,2))

                          plant_density=stand_count/canopy_cover_area_acres
                          plant_density=float(np.round(plant_density,1))

                          recommended_plants=total_area_m2/spacing
                          recommended_plants=int(np.round(recommended_plants))

                          planned_seedling_density= recommended_plants/canopy_cover_area_acres
                          planned_seedling_density=float(np.round(planned_seedling_density,1))

                          recommended_plants=int(np.round(recommended_plants))
                          stand_count=int(np.round(stand_count))


                          difference=recommended_plants-stand_count
                          under_norm=difference/recommended_plants
                          under_norm=area_converter(under_norm)*100

                          #bar chart
                          labels = ['Recommended7 Plants', 'Stand Count']
                          values = [recommended_plants, stand_count]
                          colors = ['green', 'blue']
                          # plt.figure(figsize=(7,5))
                          # ax = sns.barplot(x=labels, y=values, palette=colors)
                          # plt.title('Stand Cout Analysis')
                          # plt.yticks([])
                          # for i, value in enumerate(values):
                          #     ax.text(i, value + 0.5, f'{value:,}', ha='center', va='bottom')
                          # plt.savefig(f"output/potato_stand_count/stand_count_bar_chart_{save_date}.jpg")
                          # plt.close()

                          # creating the map
                          pred_df= pd.DataFrame(argument,columns=['pred'])
                          df_lat_long=pd.concat([df['Longitude'],df['Latitude'],pred_df['pred']],axis=1)
                          geometry = [Point(xy) for xy in zip(df_lat_long ['Longitude'], df_lat_long ['Latitude'])]
                          gdf = gpd.GeoDataFrame(df_lat_long , crs="EPSG:4326", geometry=geometry)
                          # print(gdf)
                          colors = {0: 'orange', 1: 'purple'}
                          labels={'Plants':'orange','No Plants':'purple'}
                          ax = gdf.plot(color=[colors[val] for val in gdf['pred']], legend=True)

                          plt.title('Stand Count Map')
                          ax.set_xlabel('')
                          ax.set_ylabel('')
                          ax.set_xticks([])
                          ax.set_yticks([])

                          handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=15, label=label) for label, color in labels.items()]
                          ax.legend(handles=handles, title='Legend', title_fontsize='large', loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True, shadow=True, ncol=2)

                          fig = plt.gcf()
                          fig.set_size_inches(6, 6)
                          map_rel_path=cc_output_directory+"/stand_count_map_"
                          map_path = f"{map_rel_path}{save_date}.jpg"
                          plt.savefig(map_path,dpi=500)
                          plt.close()

                          #json file path
                          output_rel_path=cc_output_directory+"/stand_count_output_"
                          canopy_cover_output_json_path=f"{output_rel_path}{save_date}.json"

                          # date conversion dd-mm-yy
                          # analysis_date=datetime.strptime(analysis_date,'%Y-%m-%d').strftime('%d-%m-%Y')
                          sowing_date=datetime.strptime(sowing_date,'%Y-%m-%d').strftime('%d-%m-%Y')
                          all_data={
                                      "reportFor":"plant_stand_count",
                                      "name": org_name,
                                      "crop_name": crop_name,
                                      "service Name": service_name,
                                      "analysis_date": analysis_date,
                                      "sowing_date": sowing_date,
                                      "crop_area": area_converter(total_area_acres),
                                      "PLANTS COUNTED": stand_count,
                                      "PLANT DENSITY PER ACRE": plant_density,
                                      "PLANNED SEEDLING DENSITY PER ACRE": planned_seedling_density,
                                      "plants in percentages": f"{under_norm}%",
                                      "The total Plants under the norm is": difference,
                                      "bar":[{
                                      "rowKey":"Recomended Plant",
                                      "rowvalue":recommended_plants
                                      },{
                                      "rowKey":"Stand Plant",
                                      "rowvalue":stand_count
                                      }],
                                      "map":f"stand_count_map_{save_date}.jpg"
                          }


                      with open(canopy_cover_output_json_path, 'w') as json_file:
                          json.dump(all_data, json_file, indent=4)

                      # cleared the tiff file from folder
                      def clear_directory(directory_path):
                          shutil.rmtree(directory_path, ignore_errors=True)
                          os.makedirs(directory_path)

                      directory_path_to_clear=config_data_main["data_output_folder"]
                      clear_directory(directory_path_to_clear)

                      return canopy_cover_output_json_path

                    except Exception as e:
                        print(f"Error reading TIFF file: {e}")

    except Exception as e:
        print(f"Error in main processing ErroR Might be due to Satellite image or directory in which image is getting stored : {e}")


def main(org_name,crop_name,sowing_date,service_name,analysis_date,polygon_c,input_spacing):

  try:

    # org_name ='abc'
    # crop_name=data['crop_name']
    # sowing_date=data['showing_date']
    # service_name=data['service_name']
    # analysis_date=data['analysis_date']
    # polygon_c=data['geom']
    # input_spacing=data['spacing']
    stand_count_output=standcount(org_name,crop_name,sowing_date,service_name,analysis_date,polygon_c,input_spacing)
    print(stand_count_output)

  except:
      print("Service Unavailable at this moment")

if __name__ == "__main__":

    input_spacing="0.15*0.20"
    analysis_date = "2023-12-25"
    polygon_c=[[73.09556979663124, 23.8405276975719],
            [73.10241918097321, 23.84029256809925],
            [73.10268407569515, 23.83403065470798],
            [73.0957683354565, 23.83407320230542],
            [73.09556979663124, 23.8405276975719]]
    org_name='abc'
    crop_name="potato"
    sowing_date='2023-12-18'
    service_name='Stand Count'
    # data = json.loads(sys.argv[1])
    try:
        main(org_name,crop_name,sowing_date,service_name,analysis_date,polygon_c,input_spacing)
    except:
        print("internal error")


ModuleNotFoundError: No module named 'memory_profiler'

In [ ]:
!pip install sentinelhub
!pip install rasterio
!pip install memory_profiler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.4/245.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6084 sha256=75c6c9322da0b19b2efe9b750886ceba7668329bd34c84f968c8cd2c9b2cce62
  Stored in directory: /root/.cache/pip/wheels/2f/a1/c8/543df0e8f5e824c3e92a432e32deb9cd89ae686095ee8cfcbe
Successfully built utm
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 32.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#  Stand Count Analysis --optimized

# Libraries

from datetime import datetime, timedelta
import os
import pandas as pd
import seaborn as sns
import sentinelhub
from sentinelhub import SHConfig
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.colors import PowerNorm
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import numpy as np
import geopandas as gpd
import sklearn
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import load_model
import json
# import base64
# from io import BytesIO
import rasterio
from rasterio.mask import mask as rio_mask
from shapely.geometry import Point,Polygon
from osgeo import gdal
import shutil
from sentinelhub import (
        CRS,
        BBox,
        DataCollection,
        DownloadRequest,
        MimeType,
        MosaickingOrder,
        SentinelHubDownloadClient,
        SentinelHubRequest,
        bbox_to_dimensions,
)

#extracting the date
def extract_start_date(analysis_date):
    e=datetime.strptime(analysis_date,'%Y-%m-%d')
    s=e-timedelta(7)
    start_date=s.strftime('%Y-%m-%d')
    return start_date

# clip the image from shapefile
def clip_and_remove_zero_pixels(file_a,path_b,polygon_c,file_path_temp):
    with rasterio.open(file_a) as src:
        polygon = Polygon(polygon_c)

        out_image, out_transform = rio_mask(src, [polygon], crop=True)

        out_meta = src.meta.copy()
        out_meta.update({"driver": "GTiff",
                        "height": out_image.shape[1],
                        "width": out_image.shape[2],
                        "transform": out_transform})

    clipped_raster_path = file_path_temp + "file.tiff"
    with rasterio.open(clipped_raster_path, "w", **out_meta) as dest:
        dest.write(out_image)

    src_ds = gdal.Open(clipped_raster_path)
    if src_ds is None:
        print("Failed to open the clipped raster file.")
        return

    cols = src_ds.RasterXSize
    rows = src_ds.RasterYSize
    bands = src_ds.RasterCount

    driver = gdal.GetDriverByName("GTiff")
    dst_ds = driver.Create(path_b, cols, rows, bands, gdal.GDT_Float32)

    dst_ds.SetProjection(src_ds.GetProjection())
    dst_ds.SetGeoTransform(src_ds.GetGeoTransform())

    for band_idx in range(1, bands + 1):
        band = src_ds.GetRasterBand(band_idx)
        band_array = band.ReadAsArray()

        mask_array = (band_array == -99999)

        masked_band_array = np.where(mask_array, np.nan, band_array)

        dst_band = dst_ds.GetRasterBand(band_idx)
        dst_band.WriteArray(masked_band_array)

    src_ds = None
    dst_ds = None

    os.remove(clipped_raster_path)

# feature creation
def ndvi(row):
    return (row['Band_8']-row['Band_4'])/(row['Band_8']+row['Band_4'])

def gci(row):
    return (row['Band_8']-row['Band_3'])/(row['Band_8']+row['Band_3'])

def lai(row):
    return (row['Band_8']-row['Band_4'])/((row['Band_8']+row['Band_4'])*2.5)

def evi(row):
    gain_factor = 2.5
    offset = 1.0
    denominator=row['Band_8'] + 6 * row['Band_4'] - 7.5 * row['Band_2'] + offset

    if denominator==0:
        evi=0
        return evi
    else:
        evi = gain_factor * (row['Band_8']-row['Band_4']) / (denominator)
        return evi

# area conversion
def area_converter(A):
  b=str(A).split('.')
  num1=b[0]+'.'+b[1][:3]
  if float(b[1][1][:3]) < 5.0:
    num1=b[0]+'.'+b[1][:3]
    num1=float(num1)
    num1=np.round(num1,2)
    return num1
  else:
    num1=b[0]+'.'+b[1][:3]
    num2=num1[:num1.find(num1.replace(num1[-1]," "))]
    num2=num2.strip()
    num2=float(num2)
    return num2

# clear the directory
def clear_directory(directory_path):
    shutil.rmtree(directory_path, ignore_errors=True)
    os.makedirs(directory_path)

def standcount(org_name,crop_name,sowing_date,service_name,analysis_date,polygon_c,input_spacing):

    with open('/content/drive/MyDrive/internship/B - with GDAL/python_services_config.json','r') as config_file:
        get_configuration = json.load(config_file)

    config_data_stand_count=get_configuration['potato_stand_count']
    config_data_main=get_configuration['main']

    config = SHConfig()

    config.sh_client_id = config_data_main["sh_client_id"]
    config.sh_client_secret = config_data_main["sh_client_secret"]
    config.save()

    today=datetime.now()
    save_date=today.strftime("%Y-%m-%d-%H-%m-%s")

    home_directory = os.path.expanduser("~")
    cc_output_directory = os.path.join(home_directory,config_data_stand_count['output_folder'])
    os.makedirs(cc_output_directory, exist_ok=True)

    start_date=extract_start_date(analysis_date)
    end_date=analysis_date

    # extract the spacing format 25*26m2
    number=input_spacing.split("*")
    num1=float(number[0])
    num2=float(number[1])
    spacing=num1+num2

    # boudning box
    min_x = min(p[0] for p in polygon_c)
    max_x = max(p[0] for p in polygon_c)
    min_y = min(p[1] for p in polygon_c)
    max_y = max(p[1] for p in polygon_c)


    bounding_box = (min_x, min_y, max_x, max_y)
    resolution = 10
    betsiboka_bbox = BBox(bbox=bounding_box, crs=CRS.WGS84)
    betsiboka_size = bbox_to_dimensions(betsiboka_bbox, resolution=resolution)

    # print(f"Image shape at {resolution} m resolution: {betsiboka_size} pixels")

    evalscript_all_bands = """
        //VERSION=3
        function setup() {
            return {
                input: [{
                    bands: ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"],
                    units: "DN"
                }],
                output: {
                    bands: 13,
                    sampleType: "INT16"
                }
            };
        }

        function evaluatePixel(sample) {
            return [sample.B01,
                    sample.B02,
                    sample.B03,
                    sample.B04,
                    sample.B05,
                    sample.B06,
                    sample.B07,
                    sample.B08,
                    sample.B8A,
                    sample.B09,
                    sample.B10,
                    sample.B11,
                    sample.B12];
        }
    """

    request_all_bands = SentinelHubRequest(
        evalscript=evalscript_all_bands,
        data_folder=config_data_main["data_output_folder"],
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L1C,
                time_interval=(str(start_date),str(end_date)),
                mosaicking_order=MosaickingOrder.LEAST_CC,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=betsiboka_bbox,
        size=betsiboka_size,
        config=config,
    )

    try:

        all_bands_response = request_all_bands.get_data()

        # stacked image
        # %%time
        all_bands_img = request_all_bands.get_data(save_data=True)

        for folder, _, filenames in os.walk(request_all_bands.data_folder):
            for filename in filenames:
                file_path = os.path.join(folder, filename)

                if filename.endswith(".tif") or filename.endswith(".tiff"):

                    try:
                      clip_and_remove_zero_pixels(file_path,file_path,polygon_c,config_data_main["data_output_folder"])

                      # main functionality on file path
                      selected_bands=[2,3,4,8]
                      with rasterio.open(file_path) as src:
                          bands_data = src.read(selected_bands)
                          width, height = src.width, src.height
                          count = len(selected_bands)
                          flat_data = bands_data.reshape((count, -1)).T
                          column_names = ['Band_{}'.format(band) for band in selected_bands]
                          transform = src.transform
                          srs = src.crs
                          rows, cols = src.height, src.width
                          y_coords, x_coords = [y for y in range(rows) for _ in range(cols)], [x for _ in range(rows) for x in range(cols)]
                          pixel_coords = list(zip(y_coords, x_coords))
                          lonlat_coords = [src.transform * (x, y) for y, x in pixel_coords]
                          pixel_values_with_coords = [(lon, lat, *values) for (lon, lat), values in zip(lonlat_coords, flat_data)]
                          column_names = ['Longitude', 'Latitude'] + column_names
                          df = pd.DataFrame(pixel_values_with_coords, columns=column_names)

                          for column in df.columns:
                              df = df[df[column] != 0]

                          # print(df.shape)
                          df=df.reset_index()
                          df=df.drop(columns={"index"})

                          def ndvi(row):
                              return (row['Band_8']-row['Band_4'])/(row['Band_8']+row['Band_4'])

                          def gci(row):
                              return (row['Band_8']-row['Band_3'])/(row['Band_8']+row['Band_3'])

                          def lai(row):
                              return (row['Band_8']-row['Band_4'])/((row['Band_8']+row['Band_4'])*2.5)

                          def evi(row):
                              gain_factor = 2.5
                              offset = 1.0
                              denominator=row['Band_8'] + 6 * row['Band_4'] - 7.5 * row['Band_2'] + offset

                              if denominator==0:
                                  evi=0
                                  return evi
                              else:
                                  evi = gain_factor * (row['Band_8']-row['Band_4']) / (denominator)
                                  return evi


                          df['ndvi']=df.apply(ndvi,axis=1)
                          df['gci']=df.apply(gci,axis=1)
                          df['lai']=df.apply(lai,axis=1)
                          df['evi']=df.apply(evi,axis=1)

                          df.columns=["Longitude","Latitude","Band1","Band2","Band3","Band4","NDVI","GCI","LAI","EVI"]

                          X=df.iloc[:,2:]

                          X[np.isinf(X)] = np.finfo(np.float32).max
                          X[np.isnan(X)] = 0

                          scaler = StandardScaler()
                          X_test = scaler.fit_transform(X)

                          #load the model
                          # classes - 0 - canopy cover , 1 - no canopy

                          #predictions
                          ann_sc = tf.keras.models.load_model(config_data_stand_count["sc_ml_model"])

                          # predictions
                          predictions_ann=ann_sc.predict(X_test)
                          argument=np.argmax(predictions_ann,axis=1)
                          total_1=np.count_nonzero(argument==1)
                          total_0=np.count_nonzero(argument==0)


                          pixel_resolution=10*10
                          total_pixel_counts=total_0+total_1
                          total_area_m2=total_pixel_counts*pixel_resolution
                          total_area_acres=total_area_m2/4046.86
                          canopy_cover_area_m2=total_0*pixel_resolution
                          canopy_cover_area_acres=canopy_cover_area_m2/4046.86

                          #calculations



                          # back up -- >>
                          # def restrict_round(c):
                          #   c=str(c)
                          #   b=c[c.find('.')-c.find('.'):c.find('.')+2]
                          #   b=float(b)
                          #   return b

                          canopy_cover_area_acres=area_converter(canopy_cover_area_acres)
                          stand_count=canopy_cover_area_m2/spacing
                          stand_count=int(np.round(stand_count,2))

                          plant_density=stand_count/canopy_cover_area_acres
                          plant_density=float(np.round(plant_density,1))

                          recommended_plants=total_area_m2/spacing
                          recommended_plants=int(np.round(recommended_plants))

                          planned_seedling_density= recommended_plants/canopy_cover_area_acres
                          planned_seedling_density=float(np.round(planned_seedling_density,1))

                          recommended_plants=int(np.round(recommended_plants))
                          stand_count=int(np.round(stand_count))


                          difference=recommended_plants-stand_count
                          under_norm=difference/recommended_plants
                          under_norm=area_converter(under_norm)*100

                          #bar chart
                          labels = ['Recommended7 Plants', 'Stand Count']
                          values = [recommended_plants, stand_count]
                          colors = ['green', 'blue']
                          # plt.figure(figsize=(7,5))
                          # ax = sns.barplot(x=labels, y=values, palette=colors)
                          # plt.title('Stand Cout Analysis')
                          # plt.yticks([])
                          # for i, value in enumerate(values):
                          #     ax.text(i, value + 0.5, f'{value:,}', ha='center', va='bottom')
                          # plt.savefig(f"output/potato_stand_count/stand_count_bar_chart_{save_date}.jpg")
                          # plt.close()

                          # creating the map
                          pred_df= pd.DataFrame(argument,columns=['pred'])
                          df_lat_long=pd.concat([df['Longitude'],df['Latitude'],pred_df['pred']],axis=1)
                          geometry = [Point(xy) for xy in zip(df_lat_long ['Longitude'], df_lat_long ['Latitude'])]
                          gdf = gpd.GeoDataFrame(df_lat_long , crs="EPSG:4326", geometry=geometry)
                          # print(gdf)
                          colors = {0: 'orange', 1: 'purple'}
                          labels={'Plants':'orange','No Plants':'purple'}
                          ax = gdf.plot(color=[colors[val] for val in gdf['pred']], legend=True, marker='s', markersize=180)

                          plt.title('Stand Count Map')
                          ax.set_xlabel('')
                          ax.set_ylabel('')
                          ax.set_xticks([])
                          ax.set_yticks([])

                          handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=15, label=label) for label, color in labels.items()]
                          ax.legend(handles=handles, title='Legend', title_fontsize='large', loc='upper center', bbox_to_anchor=(0.5, -0.01), fancybox=True, shadow=True, ncol=2)

                          fig = plt.gcf()
                          fig.set_size_inches(6, 6)
                          map_rel_path=cc_output_directory+"/stand_count_map_"
                          map_path = "{}{}.jpg".format(map_rel_path,save_date)
                          plt.savefig(map_path,dpi=500)
                          plt.close()

                          #json file path
                          output_rel_path=cc_output_directory+"/stand_count_output_"
                          stand_count_output_json_path="{}{}.json".format(output_rel_path,save_date)

                          # date conversion dd-mm-yy
                          # analysis_date=datetime.strptime(analysis_date,'%Y-%m-%d').strftime('%d-%m-%Y')
                          sowing_date=datetime.strptime(sowing_date,'%Y-%m-%d').strftime('%d-%m-%Y')
                          all_data={
                                      "reportFor":"plant_stand_count",
                                      "name": org_name,
                                      "crop_name": crop_name,
                                      "service Name": service_name,
                                      "analysis_date": analysis_date,
                                      "sowing_date": sowing_date,
                                      "crop_area": area_converter(total_area_acres),
                                      "PLANTS COUNTED": stand_count,
                                      "PLANT DENSITY PER ACRE": plant_density,
                                      "PLANNED SEEDLING DENSITY PER ACRE": planned_seedling_density,
                                      "plants in percentages": "{}%".format(under_norm),
                                      "The total Plants under the norm is": difference,
                                      "bar":[{
                                      "rowKey":"Recomended Plant",
                                      "rowvalue":recommended_plants
                                      },{
                                      "rowKey":"Stand Plant",
                                      "rowvalue":stand_count
                                      }],
                                      "map":"{}{}{}.jpg".format('stand_count','/stand_count_map_',save_date)
                          }


                      with open(stand_count_output_json_path, 'w') as json_file:
                          json.dump(all_data, json_file, indent=4)

                      # cleared the tiff file from folder
                      def clear_directory(directory_path):
                          shutil.rmtree(directory_path, ignore_errors=True)
                          os.makedirs(directory_path)

                      directory_path_to_clear=config_data_main["data_output_folder"]
                      clear_directory(directory_path_to_clear)

                      return stand_count_output_json_path

                    except Exception as e:
                        print("Error reading TIFF file: {}".format(e))

    except Exception as e:
        print("Error in main processing ErroR Might be due to Satellite image or directory in which image is getting stored : {}".format(e))


def main(org_name,crop_name,sowing_date,service_name,analysis_date,polygon_c,input_spacing):

  try:

    # org_name ='abc'
    # crop_name=data['crop_name']
    # sowing_date=data['showing_date']
    # service_name=data['service_name']
    # analysis_date=data['analysis_date']
    # polygon_c=data['geom']
    # input_spacing=data['spacing']
    stand_count_output=standcount(org_name,crop_name,sowing_date,service_name,analysis_date,polygon_c,input_spacing)
    print(stand_count_output)

  except:
      print("Service Unavailable at this moment")

if __name__ == "__main__":

    input_spacing="0.15*0.20"
    analysis_date = "2023-12-25"
    polygon_c=[[73.09556979663124, 23.8405276975719],
            [73.10241918097321, 23.84029256809925],
            [73.10268407569515, 23.83403065470798],
            [73.0957683354565, 23.83407320230542],
            [73.09556979663124, 23.8405276975719]]
    org_name='abc'
    crop_name="potato"
    sowing_date='2023-12-18'
    service_name='Stand Count'
    # data = json.loads(sys.argv[1])
    try:
        %load_ext memory_profiler
        # %memit -r 1
        %memit -r 1 main(org_name,crop_name,sowing_date,service_name,analysis_date,polygon_c,input_spacing)
    except:
        print("internal error")


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


154/154 [==============================] - 0s 2ms/step
/root/agri_crop/image/potato/stand_count/stand_count_output_2024-04-14-04-04-1713068175.json
peak memory: 895.26 MiB, increment: 66.17 MiB


In [ ]:
#  Stand Count Analysis

# Libraries

from datetime import datetime, timedelta
import os
import pandas as pd
import seaborn as sns
import sentinelhub
from sentinelhub import SHConfig
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.colors import PowerNorm
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import numpy as np
import geopandas as gpd
import sklearn
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import load_model
import json
import memory_profiler
import rasterio
from rasterio.mask import mask as rio_mask
from shapely.geometry import Point,Polygon
from osgeo import gdal
import shutil
from sentinelhub import (
        CRS,
        BBox,
        DataCollection,
        DownloadRequest,
        MimeType,
        MosaickingOrder,
        SentinelHubDownloadClient,
        SentinelHubRequest,
        bbox_to_dimensions,
)


def standcount(org_name,crop_name,sowing_date,service_name,analysis_date,polygon_c,input_spacing):

    with open('/content/drive/MyDrive/internship/B - with GDAL/python_services_config.json','r') as config_file:
        get_configuration = json.load(config_file)

    config_data_stand_count=get_configuration['potato_stand_count']
    config_data_main=get_configuration['main']

    config = SHConfig()

    config.sh_client_id = config_data_main["sh_client_id"]
    config.sh_client_secret = config_data_main["sh_client_secret"]
    config.save()

    today=datetime.now()
    save_date=today.strftime("%Y-%m-%d-%H-%m-%s")

    home_directory = os.path.expanduser("~")
    cc_output_directory = os.path.join(home_directory,config_data_stand_count['output_folder'])
    os.makedirs(cc_output_directory, exist_ok=True)

    # date
    def extract_start_date(analysis_date):
        e=datetime.strptime(analysis_date,'%Y-%m-%d')
        s=e-timedelta(7)
        start_date=s.strftime('%Y-%m-%d')
        return start_date

    start_date=extract_start_date(analysis_date)
    end_date=analysis_date

    # extract the spacing format 25*26m2
    number=input_spacing.split("*")
    num1=float(number[0])
    num2=float(number[1])
    spacing=num1+num2

    # boudning box
    min_x = min(p[0] for p in polygon_c)
    max_x = max(p[0] for p in polygon_c)
    min_y = min(p[1] for p in polygon_c)
    max_y = max(p[1] for p in polygon_c)


    bounding_box = (min_x, min_y, max_x, max_y)
    resolution = 10
    betsiboka_bbox = BBox(bbox=bounding_box, crs=CRS.WGS84)
    betsiboka_size = bbox_to_dimensions(betsiboka_bbox, resolution=resolution)

    # print(f"Image shape at {resolution} m resolution: {betsiboka_size} pixels")

    evalscript_all_bands = """
        //VERSION=3
        function setup() {
            return {
                input: [{
                    bands: ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"],
                    units: "DN"
                }],
                output: {
                    bands: 13,
                    sampleType: "INT16"
                }
            };
        }

        function evaluatePixel(sample) {
            return [sample.B01,
                    sample.B02,
                    sample.B03,
                    sample.B04,
                    sample.B05,
                    sample.B06,
                    sample.B07,
                    sample.B08,
                    sample.B8A,
                    sample.B09,
                    sample.B10,
                    sample.B11,
                    sample.B12];
        }
    """

    request_all_bands = SentinelHubRequest(
        evalscript=evalscript_all_bands,
        data_folder=config_data_main["data_output_folder"],
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L1C,
                time_interval=(str(start_date),str(end_date)),
                mosaicking_order=MosaickingOrder.LEAST_CC,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=betsiboka_bbox,
        size=betsiboka_size,
        config=config,
    )

    try:

        all_bands_response = request_all_bands.get_data()

        # stacked image
        # %%time
        all_bands_img = request_all_bands.get_data(save_data=True)

        for folder, _, filenames in os.walk(request_all_bands.data_folder):
            for filename in filenames:
                file_path = os.path.join(folder, filename)

                if filename.endswith(".tif") or filename.endswith(".tiff"):

                    try:
                      # clip the raster for getting the image with nan pixels
                      selected_bands=[2,3,4,8]
                      def clip_and_remove_zero_pixels(file_a,path_b,polygon_c):
                          with rasterio.open(file_a) as src:
                              polygon = Polygon(polygon_c)

                              out_image, out_transform = rio_mask(src, [polygon], crop=True)

                              out_meta = src.meta.copy()
                              out_meta.update({"driver": "GTiff",
                                              "height": out_image.shape[1],
                                              "width": out_image.shape[2],
                                              "transform": out_transform})

                          clipped_raster_path = config_data_main["data_output_folder"]+"file.tiff"
                          with rasterio.open(clipped_raster_path, "w", **out_meta) as dest:
                              dest.write(out_image)

                          src_ds = gdal.Open(clipped_raster_path)
                          if src_ds is None:
                              print("Failed to open the clipped raster file.")
                              return

                          cols = src_ds.RasterXSize
                          rows = src_ds.RasterYSize
                          bands = src_ds.RasterCount

                          driver = gdal.GetDriverByName("GTiff")
                          dst_ds = driver.Create(path_b, cols, rows, bands, gdal.GDT_Float32)

                          dst_ds.SetProjection(src_ds.GetProjection())
                          dst_ds.SetGeoTransform(src_ds.GetGeoTransform())

                          for band_idx in range(1, bands + 1):
                              band = src_ds.GetRasterBand(band_idx)
                              band_array = band.ReadAsArray()

                              mask_array = (band_array == -99999)

                              masked_band_array = np.where(mask_array, np.nan, band_array)

                              dst_band = dst_ds.GetRasterBand(band_idx)
                              dst_band.WriteArray(masked_band_array)
                          src_ds = None
                          dst_ds = None
                          os.remove(clipped_raster_path)

                      clip_and_remove_zero_pixels(file_path,file_path,polygon_c)

                      # main functionality on file path
                      selected_bands=[2,3,4,8]
                      with rasterio.open(file_path) as src:
                          bands_data = src.read(selected_bands)
                          width, height = src.width, src.height
                          count = len(selected_bands)
                          flat_data = bands_data.reshape((count, -1)).T
                          column_names = [f'Band_{band}' for band in selected_bands]
                          transform = src.transform
                          srs = src.crs
                          rows, cols = src.height, src.width
                          y_coords, x_coords = [y for y in range(rows) for _ in range(cols)], [x for _ in range(rows) for x in range(cols)]
                          pixel_coords = list(zip(y_coords, x_coords))
                          lonlat_coords = [src.transform * (x, y) for y, x in pixel_coords]
                          pixel_values_with_coords = [(lon, lat, *values) for (lon, lat), values in zip(lonlat_coords, flat_data)]
                          column_names = ['Longitude', 'Latitude'] + column_names
                          df = pd.DataFrame(pixel_values_with_coords, columns=column_names)

                          for column in df.columns:
                              df = df[df[column] != 0]

                          # print(df.shape)
                          df=df.reset_index()
                          df=df.drop(columns={"index"})


                          df.columns=["Longitude","Latitude","Band1","Band2","Band3","Band4"]

                          X=df.iloc[:,2:]

                          X[np.isinf(X)] = np.finfo(np.float32).max
                          X[np.isnan(X)] = 0

                          scaler = StandardScaler()
                          X_test = scaler.fit_transform(X)

                          #load the model
                          # classes - 0 - canopy cover , 1 - no canopy

                          #predictions
                          ann_sc = tf.keras.models.load_model(config_data_stand_count["sc_ml_model"])

                          # predictions
                          predictions_ann=ann_sc.predict(X_test)
                          # argument=np.argmax(predictions_ann,axis=1)
                          argument=np.where(predictions_ann < 0.5, 0, 1)

                          total_1=np.count_nonzero(argument==1)
                          total_0=np.count_nonzero(argument==0)

                          pixel_resolution=10*10
                          total_pixel_counts=total_0+total_1
                          total_area_m2=total_pixel_counts*pixel_resolution
                          total_area_acres=total_area_m2/4046.86
                          canopy_cover_area_m2=total_0*pixel_resolution
                          canopy_cover_area_acres=canopy_cover_area_m2/4046.86

                          #calculations

                          def area_converter(A):
                            b=str(A).split('.')
                            num1=b[0]+'.'+b[1][:3]
                            if float(b[1][1][:3]) < 5.0:
                              num1=b[0]+'.'+b[1][:3]
                              num1=float(num1)
                              num1=np.round(num1,2)
                              return num1
                            else:
                              num1=b[0]+'.'+b[1][:3]
                              num2=num1[:num1.find(num1.replace(num1[-1]," "))]
                              num2=num2.strip()
                              num2=float(num2)
                              return num2

                          # back up -- >>
                          # def restrict_round(c):
                          #   c=str(c)
                          #   b=c[c.find('.')-c.find('.'):c.find('.')+2]
                          #   b=float(b)
                          #   return b

                          canopy_cover_area_acres=area_converter(canopy_cover_area_acres)
                          stand_count=canopy_cover_area_m2/spacing
                          stand_count=int(np.round(stand_count,2))

                          plant_density=stand_count/canopy_cover_area_acres
                          plant_density=float(np.round(plant_density,1))

                          recommended_plants=total_area_m2/spacing
                          recommended_plants=int(np.round(recommended_plants))

                          planned_seedling_density= recommended_plants/canopy_cover_area_acres
                          planned_seedling_density=float(np.round(planned_seedling_density,1))

                          recommended_plants=int(np.round(recommended_plants))
                          stand_count=int(np.round(stand_count))


                          difference=recommended_plants-stand_count
                          under_norm=difference/recommended_plants
                          under_norm=area_converter(under_norm)*100

                          #bar chart
                          labels = ['Recommended7 Plants', 'Stand Count']
                          values = [recommended_plants, stand_count]
                          colors = ['green', 'blue']
                          # plt.figure(figsize=(7,5))
                          # ax = sns.barplot(x=labels, y=values, palette=colors)
                          # plt.title('Stand Cout Analysis')
                          # plt.yticks([])
                          # for i, value in enumerate(values):
                          #     ax.text(i, value + 0.5, f'{value:,}', ha='center', va='bottom')
                          # plt.savefig(f"output/potato_stand_count/stand_count_bar_chart_{save_date}.jpg")
                          # plt.close()

                          # creating the map
                          pred_df= pd.DataFrame(argument,columns=['pred'])
                          df_lat_long=pd.concat([df['Longitude'],df['Latitude'],pred_df['pred']],axis=1)
                          geometry = [Point(xy) for xy in zip(df_lat_long ['Longitude'], df_lat_long ['Latitude'])]
                          gdf = gpd.GeoDataFrame(df_lat_long , crs="EPSG:4326", geometry=geometry)
                          # print(gdf)
                          colors = {0: 'orange', 1: 'purple'}
                          labels={'Plants':'orange','No Plants':'purple'}
                          ax = gdf.plot(color=[colors[val] for val in gdf['pred']], legend=True)

                          plt.title('Stand Count Map')
                          ax.set_xlabel('')
                          ax.set_ylabel('')
                          ax.set_xticks([])
                          ax.set_yticks([])

                          handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=15, label=label) for label, color in labels.items()]
                          ax.legend(handles=handles, title='Legend', title_fontsize='large', loc='upper center', bbox_to_anchor=(0.5, -0.01), fancybox=True, shadow=True, ncol=2)

                          fig = plt.gcf()
                          fig.set_size_inches(6, 6)
                          map_rel_path=cc_output_directory+"/stand_count_map_"
                          map_path = f"{map_rel_path}{save_date}.jpg"
                          plt.savefig(map_path,dpi=500)
                          plt.close()

                          #json file path
                          output_rel_path=cc_output_directory+"/stand_count_output_"
                          canopy_cover_output_json_path=f"{output_rel_path}{save_date}.json"

                          # date conversion dd-mm-yy
                          # analysis_date=datetime.strptime(analysis_date,'%Y-%m-%d').strftime('%d-%m-%Y')
                          sowing_date=datetime.strptime(sowing_date,'%Y-%m-%d').strftime('%d-%m-%Y')
                          all_data={
                                      "reportFor":"plant_stand_count",
                                      "name": org_name,
                                      "crop_name": crop_name,
                                      "service Name": service_name,
                                      "analysis_date": analysis_date,
                                      "sowing_date": sowing_date,
                                      "crop_area": area_converter(total_area_acres),
                                      "PLANTS COUNTED": stand_count,
                                      "PLANT DENSITY PER ACRE": plant_density,
                                      "PLANNED SEEDLING DENSITY PER ACRE": planned_seedling_density,
                                      "plants in percentages": f"{under_norm}%",
                                      "The total Plants under the norm is": difference,
                                      "bar":[{
                                      "rowKey":"Recomended Plant",
                                      "rowvalue":recommended_plants
                                      },{
                                      "rowKey":"Stand Plant",
                                      "rowvalue":stand_count
                                      }],
                                      "map":f"stand_count_map_{save_date}.jpg"
                          }


                      with open(canopy_cover_output_json_path, 'w') as json_file:
                          json.dump(all_data, json_file, indent=4)

                      # cleared the tiff file from folder
                      def clear_directory(directory_path):
                          shutil.rmtree(directory_path, ignore_errors=True)
                          os.makedirs(directory_path)

                      directory_path_to_clear=config_data_main["data_output_folder"]
                      clear_directory(directory_path_to_clear)

                      return canopy_cover_output_json_path

                    except Exception as e:
                        print(f"Error reading TIFF file: {e}")

    except Exception as e:
        print(f"Error in main processing ErroR Might be due to Satellite image or directory in which image is getting stored : {e}")


def main(org_name,crop_name,sowing_date,service_name,analysis_date,polygon_c,input_spacing):

  try:

    # org_name ='abc'
    # crop_name=data['crop_name']
    # sowing_date=data['showing_date']
    # service_name=data['service_name']
    # analysis_date=data['analysis_date']
    # polygon_c=data['geom']
    # input_spacing=data['spacing']
    stand_count_output=standcount(org_name,crop_name,sowing_date,service_name,analysis_date,polygon_c,input_spacing)
    print(stand_count_output)

  except:
      print("Service Unavailable at this moment")

if __name__ == "__main__":

    input_spacing="0.15*0.20"
    analysis_date = "2023-12-25"
    polygon_c=[[73.09556979663124, 23.8405276975719],
            [73.10241918097321, 23.84029256809925],
            [73.10268407569515, 23.83403065470798],
            [73.0957683354565, 23.83407320230542],
            [73.09556979663124, 23.8405276975719]]
    org_name='abc'
    crop_name="potato"
    sowing_date='2023-12-18'
    service_name='Stand Count'
    # data = json.loads(sys.argv[1])
    try:
        main(org_name,crop_name,sowing_date,service_name,analysis_date,polygon_c,input_spacing)
    except:
        print("internal error")


154/154 [==============================] - 0s 2ms/step
/root/agri_crop/image/potato/stand_count/stand_count_output_2024-04-22-10-04-1713782452.json
